In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [7]:
###Set constants and parameters

##input params
H = 0.42 #no units,ratio #Hematocrit assumed to be constant
M = 20e-9 #mol_ O2/(mL/s) #Taken from table 2 from Wiley Payne paper
cHb = 0.2 #mL_O2/m #Taken from table 2 from Wiley Payne paper
paO2_bar_t = 15 #mmHG #Taken from table 2 from Wiley Payne paper

##SI them
M = M*10e3
cHb = cHb*10e-3
paO2_bar_t = paO2_bar_t*133.322

###Set up input data

##reset df
df = None

##Input data
#all data taken from Payne article
df = pd.DataFrame({'Name': ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'C', 'V6', 'V5', 'V4', 'V3', 'V2', 'V1'], 
                   'Number': [1, 2, 4, 8, 16, 32, 64, 32, 16, 8, 4, 2, 1],
                   'Diameter(µm)': [23.97, 19.17, 15.28, 12.08, 9.46, 7.32, 8, 11.51, 14.53, 17.79, 21.45, 25.70, 30.77],
                   'Length(µm)': [1267.6, 930.3, 543.6, 302.3, 161.2, 154.7, 243.9, 473.9, 272.3, 426.6, 632.5, 844.2, 936.3],
                   'Viscocity(mPAs)': [1.59, 1.50, 1.42, 1.34, 1.28, 1.23, 1.24, 1.33, 1.40, 1.48, 1.55, 1.62, 1.7],
                   'Pressure Drop(mmHg)': [6.93, 5.87, 4.02, 2.70, 1.82, 2.35, 2.62, 1.27, 0.61, 0.89, 1.31, 1.78, 2.01],
                   'Saturation': [0.94, 0.93, 0.92, 0.89, 0.84, 0.765, 0.665, 0.61, 0.5975, 0.5875, 0.5825, 0.5775, 0.5725]
                  })

##change units
df['Diameter(µm)'] = df['Diameter(µm)']*1e-6
df['Length(µm)'] = df['Length(µm)']*1e-6
df['Viscocity(mPAs)'] = df['Viscocity(mPAs)']*1e-3
df['Pressure Drop(mmHg)'] = df['Pressure Drop(mmHg)']*133.322
##rename the titles for the units
df = df.rename(columns={'Diameter(µm)':'Diameter(m)', 'Length(µm)':'Length(m)', 'Viscocity(mPAs)':'Viscocity(PAs)', 'Pressure Drop(mmHg)':'Pressure Drop(Pa)'})

###Calculate Flows

##Calculate resitances of each single vessel for each type
df['Resistance for U'] = 32 * df['Viscocity(PAs)'] * df['Length(m)'].div(df['Diameter(m)']**2)
df['Resistance for Q'] = 128 * df['Viscocity(PAs)'] * df['Length(m)'].div(math.pi*(df['Diameter(m)']**4))

##Alter resistances for C
#These are aross each pressure drop (explains the doubled resistance for the capillaries as in paralell so have half the flow each (flow is shared))
df.loc[df.index[df['Name'] == 'C'][0],'Resistance for U'] = df.loc[df.index[df['Name'] == 'C'][0],'Resistance for U']*2
df.loc[df.index[df['Name'] == 'C'][0],'Resistance for Q'] = df.loc[df.index[df['Name'] == 'C'][0],'Resistance for Q']*2

##Calculate flow in a single vessel
df['U in single(m/s)'] = df['Pressure Drop(Pa)'].div(df['Resistance for U'])
df['Q in single(m^3/s)'] = df['Pressure Drop(Pa)'].div(df['Resistance for Q'])

########temp calculate the saturations - CANT GET TO WORK
#df['Vt'] = np.zeros(13)
#S_temp = pd.DataFrame({'Saturation': np.zeros(13)})
#S_temp['Saturation'][0] = 0.94
#df = pd.concat([df,S_temp], axis=1)

#df.loc[0,'Vt'] = df.loc[0,'Q in single(m^3/s)']*cHb*H*df.loc[0,'Saturation']/M
#for i in range(12):
    #df.loc[i+1,'Saturation'] = df.loc[i,'Saturation'] - ( 2*M*df.loc[i+1,'Vt'] ) / ( cHb*H*( df.loc[i,'Q in single(m^3/s)'] + df.loc[i+1,'Q in single(m^3/s)'] ) )
    #df.loc[i+1,'Saturation'] = ( M*df.loc[i+1,'Vt'] ) / ( cHb*H*( df.loc[i,'Q in single(m^3/s)'] + df.loc[i+1,'Q in single(m^3/s)'] ) )
    #df.loc[i+1,'Vt'] = df.loc[i+1,'Q in single(m^3/s)']*cHb*H*df.loc[i+1,'Saturation']/M
    
###Next stage

##Calculate Vt
#Check what Vt is calculated in units and add to name 
df['Vt'] = df['Q in single(m^3/s)']*cHb*H*df['Saturation']/M
#This will remain fixed.


##S and partial O2 relationship given by servinghaus 1979
def check_imag_roots_real(all_roots):
    counter = 0
    sols = 0
    for i in range(len(all_roots)):
        if all_roots[i].imag <= 1e-13 and all_roots[i].imag >= -1e-13:
            sols = all_roots[i].real
            counter += 1
    if counter != 1:
        print('Something seems wrong with the partial presure to Saturation cubic solver as not one solution is returned.')
    return sols
 
for i in range(len(df['Saturation'])):
    root_temp = None
    sols = 0
    root_temp = np.roots([1,0,150,23400*df.loc[i,'Saturation']/(df.loc[i,'Saturation']-1)]) #gives mmHg , *133.322
    sols = check_imag_roots_real(root_temp) 
    #sols = sols*133.322 ###Remove this step if want in mmHg
    df.loc[i,'partial pressure blood(pa)'] = sols
#Checked partial pressure against typical values online and it seems bang on.

###Now calculate the partial pressure of O2 in tissue
#Will take my vessel average partial pressure to be the 0.5(pin + pout)
#need h
#all units were in mm so i played around with this equation from the paper
df['X-Area wall(m^2)'] = 1e-6 * np.pi * ( 0.16*(df['Diameter(m)']*1e3)**2 + 1.4*(df['Diameter(m)']*1e3) + 14 )
df['wall thickness(mm)'] = ( -(df['Diameter(m)']*1e3) + np.sqrt((df['Diameter(m)']*1e3)**2 + (4*(df['X-Area wall(m^2)']*1e6)/np.pi)) ) / 2

In [6]:
df[['X-Area wall(m^2)','wall thickness(mm)']]

,X-Area wall(m^2),wall thickness(mm)
0,0.000044,3.734186
1,0.000044,3.735677
2,0.000044,3.736888
3,0.000044,3.737885
4,0.000044,3.738702
5,0.000044,3.739370
6,0.000044,3.739157
7,0.000044,3.738062
8,0.000044,3.737121
9,0.000044,3.736106
